In [4]:
import numpy as np
from pprint import pprint

In [5]:
%%html
<h1 style = 'text-align:center;'>Program HMM<h1>
<h2>&#10004 Read <em>transition</em> (M&#215M) and <em>emission</em> (M&#215K) matrices and beginning distribution
(vector of length M) from a file (tab separated format).</h2>

In [13]:
#transition (M×M)
A = np.loadtxt("matrixA")
#emission (M×K) 
B = np.loadtxt("matrixB")
print("A:",A,"\nB:",B)

A: [[0.95 0.05]
 [0.1  0.9 ]] 
B: [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.1        0.1        0.1        0.1        0.1        0.5       ]]


In [14]:
%%html
<h2>&#10004 HMM random generator: generate a sequence x of length L given the model parameters. Output x and π in a tab-separated format.<h2>

In [72]:
def HMMGenerator(L,B,pi='random'):
    x = np.random.randint(B.shape[1], size=L)
    xSymbol = np.array(['e'+str(item) for item in x])
    if pi=='random':
        pi = np.random.dirichlet(np.ones(B.shape[0]),size=1)[0]
    print("x:",xSymbol,"\npi:",pi)
    return {'x':x,'xSymbol':xSymbol,'pi':pi}

class HiddenMarkovChain(object):
    def __init__(self, A, B, pi, states, observations):
        self.A = A
        self.B = B
        self.pi = pi
        self.states = states
        self.observations = observations
 
    def next_state(self, current_state):
        pStates = self.A[self.states.index(current_state)]
        stateNext = np.random.choice(
            self.states, 
            p = pStates
        )
        pObservations = self.B[self.states.index(stateNext)]
        observationNext = np.random.choice(
            self.observations, 
            p = pObservations
        )
        return {'state': stateNext,'observation': observationNext}
 
    def generate_states(self, current_state, no=10):
        pPi = self.pi
        firstState = np.random.choice(
            self.states, 
            p = pPi
        )
        current_state = firstState
        future_states = []
        future_observations = []
        for i in range(no):
            nextObject = self.next_state(current_state)
            next_state = nextObject['state']
            next_observation = nextObject['observation']
            future_states.append(next_state)
            future_observations.append(next_observation)
            current_state = next_state
        return future_states,future_observations

In [94]:
#HMMObject = HMMGenerator(12,B)
states = tuple([ 'p' + str(i) for i in range(B.shape[0])])
observations = tuple([ 'e' + str(i) for i in range(B.shape[1])])
pi = [0.5,0.5]
hiddenModal = HiddenMarkovChain(A,B,pi,states,observations)
HMMObject = hiddenModal.generate_states('p0',no=20)
x = [int(item[1:]) for item in HMMObject[1]]
xSymbol = HMMObject[1]
pathStates = HMMObject[0]
#x, xSymbol, pi = HMMObject['x'], HMMObject['xSymbol'], HMMObject['pi']

print("states:",states,"\nobservations:",observations,"\nx:", x,"\nxSymbol:",xSymbol,"\npathStates:",pathStates)

states: ('p0', 'p1') 
observations: ('e0', 'e1', 'e2', 'e3', 'e4', 'e5') 
x: [0, 1, 3, 2, 5, 5, 5, 1, 5, 5, 5, 4, 2, 0, 3, 4, 4, 0, 5, 2] 
xSymbol: ['e0', 'e1', 'e3', 'e2', 'e5', 'e5', 'e5', 'e1', 'e5', 'e5', 'e5', 'e4', 'e2', 'e0', 'e3', 'e4', 'e4', 'e0', 'e5', 'e2'] 
pathStates: ['p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0']


In [82]:
%%html
<h2>&#10004 Read HMM parameters and observed sequence x. Use Viterbi algorithm to find the most probable path π given x and HMM parameters. Output the path in the tab separated format.<h2>

In [97]:
def viterbi(obs_seq,pi, A, B):
    T = len(obs_seq)
    N = A.shape[0]
    delta = np.zeros((T, N))
    psi = np.zeros((T, N))
    delta[0] = pi*B[:,obs_seq[0]]
    for t in range(1, T):
        for j in range(N):
            delta[t,j] = np.max(delta[t-1]*A[:,j]) * B[j, obs_seq[t]]
            psi[t,j] = np.argmax(delta[t-1]*A[:,j])

    states = np.zeros(T, dtype=np.int32)
    states[T-1] = np.argmax(delta[T-1])
    for t in range(T-2, -1, -1):
        states[t] = psi[t+1, states[t+1]]
    return states

In [98]:
path = viterbi(x, pi, A, B)
print("x:", list(map(lambda y: observations[y], x)))
print("path:",  list(map(lambda s: states[s], path)))

x: ['e4', 'e1', 'e1', 'e0', 'e5', 'e5', 'e5', 'e1', 'e1', 'e2', 'e2', 'e3', 'e4', 'e1', 'e3', 'e5', 'e4', 'e1', 'e3', 'e5']
path: ['p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0']


In [99]:
%%html
<h2>&#10004 Find the probability P(x) using forward algorithm (store the full matrix of forward probabilities fk(xi) in order to use it for posterior decoding later on).<h2>



In [100]:
def forward(obs_seq, pi, A, B):
    T = len(obs_seq)
    N = A.shape[0]
    alpha = np.zeros((T, N))
    alpha[0] = pi*B[:,obs_seq[0]]
    for t in range(1, T):
        alpha[t] = np.inner(alpha[t-1],A) * B[:, obs_seq[t]]
    return alpha

def likelihood(alpha):
    return  alpha[-1].sum()  

In [101]:
forwardMatrix = forward(x, pi, A, B)
print("forwardRes:",forwardMatrix)
P = likelihood(forwardMatrix)
print("P:",P)

forwardRes: [[8.33333333e-02 5.00000000e-02]
 [1.36111111e-02 5.33333333e-03]
 [2.19953704e-03 6.16111111e-04]
 [3.53394290e-04 7.74453704e-05]
 [5.65994740e-05 5.25201312e-05]
 [9.39925115e-06 2.64640327e-05]
 [1.70874837e-06 1.23787773e-05]
 [3.73708303e-07 1.13117744e-06]
 [6.85969599e-08 1.05543053e-07]
 [1.17407108e-08 1.01848443e-08]
 [1.94381957e-09 1.03404310e-09]
 [3.16388458e-10 1.12502074e-10]
 [5.10323565e-11 1.32890713e-11]
 [8.19086537e-12 1.70633998e-12]
 [1.31110652e-12 2.35479252e-13]
 [2.09554192e-13 1.71520989e-13]
 [3.46087554e-14 1.75324310e-14]
 [5.62582319e-15 1.92400634e-15]
 [9.06788725e-16 2.29418803e-16]
 [1.45486705e-16 1.48577897e-16]]
P: 2.940646021471975e-16


In [102]:
%%html
<h2>&#10004 Program the backward algorithm.<h2>

In [103]:
def backward(obs_seq, A, B):
    N = A.shape[0]
    T = len(obs_seq)

    beta = np.zeros((N,T))
    beta[:,-1:] = 1

    for t in reversed(range(T-1)):
        for n in range(N):
            beta[n,t] = np.sum(beta[:,t+1] * A[n,:] * B[:, obs_seq[t+1]])

    return beta

In [104]:
backwardMatrix=backward(x, A, B)
print("backwardMatrix",backwardMatrix.T)

backwardMatrix [[2.11040075e-15 1.75635696e-15]
 [1.27873612e-14 1.71470475e-14]
 [7.51854541e-14 1.76599518e-13]
 [4.15319526e-13 1.88530584e-12]
 [1.97309845e-12 4.11649082e-12]
 [1.10820993e-11 8.73730925e-12]
 [6.73201687e-11 1.69229032e-11]
 [4.21708283e-10 1.09938131e-10]
 [2.64028622e-09 7.32592899e-10]
 [1.65150208e-08 5.08158390e-09]
 [1.03125456e-07 3.73647367e-08]
 [6.41962400e-07 2.96281815e-07]
 [3.97377931e-06 2.55613511e-06]
 [2.43430209e-05 2.38935344e-05]
 [1.46216768e-04 2.38406537e-04]
 [8.44763040e-04 4.98504784e-04]
 [5.19078704e-03 4.57768519e-03]
 [3.13611111e-02 4.50555556e-02]
 [1.83333333e-01 4.66666667e-01]
 [1.00000000e+00 1.00000000e+00]]


In [105]:
%%html
<h2>&#10004 Program posterior decoding.<h2>


In [106]:
def posterior(forwardMatrix, backwardMatrix):
    obs_prob = likelihood(forwardMatrix)
    return (np.multiply(forwardMatrix,backwardMatrix.T) / obs_prob)


In [107]:
posteriorMatrix = posterior(forwardMatrix, backwardMatrix)
print("posteriorMatrix:",posteriorMatrix)

posteriorMatrix: [[0.59805474 0.29863454]
 [0.5918774  0.31098922]
 [0.56237027 0.37000348]
 [0.49911328 0.49651746]
 [0.37976803 0.73520796]
 [0.35421956 0.7863049 ]
 [0.39118353 0.71237697]
 [0.53592267 0.42289869]
 [0.61590415 0.26293573]
 [0.6593724  0.17599922]
 [0.68167769 0.13138864]
 [0.69069685 0.11335033]
 [0.68961487 0.11551428]
 [0.67804967 0.13864468]
 [0.65191715 0.19090973]
 [0.60198893 0.29076616]
 [0.61090889 0.27292625]
 [0.59997723 0.29478956]
 [0.5653336  0.36407683]
 [0.49474402 0.50525598]]


In [108]:
%%html
<h1>Using the previous results, build the HMM for the unfair casino.<h1>


In [109]:
%%html
<h2>&#10004 Simulate occasionally unfair casino: one dice has equal probabilities of each face
and another dice has probability of P(6) = 0.5, other outcomes’ probabilities are
0.1. The probability to switch from the fair dice to the loaded dice is 0.05, the
probability to switch from the loaded to the fair dice is 0.1.<h2>


In [110]:
#transition (M×M)
A = np.loadtxt("matrixA")
#emission (M×K) 
B = np.loadtxt("matrixB")
print("A:",A,"\nB:",B)


A: [[0.95 0.05]
 [0.1  0.9 ]] 
B: [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.1        0.1        0.1        0.1        0.1        0.5       ]]


In [125]:
%%html
<h2>&#10004Compare the simulated path and Viterbi most likely path.
<h2>


In [119]:
states = tuple([ 'p' + str(i) for i in range(B.shape[0])])
observations = tuple([ 'e' + str(i) for i in range(B.shape[1])])
pi = [0.5,0.5]
hiddenModal = HiddenMarkovChain(A,B,pi,states,observations)
HMMObject = hiddenModal.generate_states('p0',no=30)
x = [int(item[1:]) for item in HMMObject[1]]
xSymbol = HMMObject[1]
pathStates = HMMObject[0]
print("states:",states,"\nobservations:",observations,"\nx:", x,"\nxSymbol:",xSymbol,"\npathStates:",pathStates)

states: ('p0', 'p1') 
observations: ('e0', 'e1', 'e2', 'e3', 'e4', 'e5') 
x: [1, 2, 5, 4, 5, 0, 5, 5, 4, 5, 4, 5, 2, 2, 3, 3, 5, 0, 5, 5, 4, 5, 4, 2, 4, 1, 2, 3, 2, 5] 
xSymbol: ['e1', 'e2', 'e5', 'e4', 'e5', 'e0', 'e5', 'e5', 'e4', 'e5', 'e4', 'e5', 'e2', 'e2', 'e3', 'e3', 'e5', 'e0', 'e5', 'e5', 'e4', 'e5', 'e4', 'e2', 'e4', 'e1', 'e2', 'e3', 'e2', 'e5'] 
pathStates: ['p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p0', 'p0', 'p0', 'p0', 'p0', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0']


In [120]:
path = viterbi(x, pi, A, B)

x: ['e1', 'e2', 'e5', 'e4', 'e5', 'e0', 'e5', 'e5', 'e4', 'e5', 'e4', 'e5', 'e2', 'e2', 'e3', 'e3', 'e5', 'e0', 'e5', 'e5', 'e4', 'e5', 'e4', 'e2', 'e4', 'e1', 'e2', 'e3', 'e2', 'e5']
path: ['p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0']


In [123]:
print("x:", list(map(lambda y: observations[y], x)))
print("Viterbi path:",  list(map(lambda s: states[s], path)))
print("Real path:",  pathStates)

x: ['e1', 'e2', 'e5', 'e4', 'e5', 'e0', 'e5', 'e5', 'e4', 'e5', 'e4', 'e5', 'e2', 'e2', 'e3', 'e3', 'e5', 'e0', 'e5', 'e5', 'e4', 'e5', 'e4', 'e2', 'e4', 'e1', 'e2', 'e3', 'e2', 'e5']
Viterbi path: ['p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0']
Real path: ['p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p0', 'p0', 'p0', 'p0', 'p0', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p1', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0', 'p0']


In [126]:
%%html
<h2>&#10004 Plot posterior probability P(f air). Compare with the regions of the simulated
path where loaded dice was used.
<h2>


In [127]:
forwardMatrix = forward(x, pi, A, B)
print("forwardRes:",forwardMatrix)
P = likelihood(forwardMatrix)
print("P:",P)

forwardRes: [[8.33333333e-02 5.00000000e-02]
 [1.36111111e-02 5.33333333e-03]
 [2.19953704e-03 3.08055556e-03]
 [3.73931327e-04 2.99245370e-04]
 [6.16995049e-05 1.53356983e-04]
 [1.10470631e-05 1.44191235e-05]
 [1.86927769e-06 7.04095874e-06]
 [3.54643624e-07 3.26189532e-06]
 [8.33343681e-08 2.97117015e-07]
 [1.56705834e-08 1.37869375e-07]
 [3.63008717e-09 1.25649496e-08]
 [6.79471715e-10 5.83573168e-09]
 [1.56214119e-10 5.32010568e-10]
 [2.91673235e-11 4.94430923e-11]
 [5.03018533e-12 4.74155154e-12]
 [8.35958940e-13 4.77041492e-13]
 [1.36335511e-13 2.56466618e-13]
 [2.37236778e-14 2.44453508e-14]
 [3.95996024e-15 1.21865917e-14]
 [7.28548635e-16 5.68196429e-15]
 [1.62703236e-16 5.18662273e-16]
 [3.00835314e-17 2.41533185e-16]
 [6.77600234e-18 2.20388219e-17]
 [1.25652389e-18 2.05125400e-18]
 [2.16043399e-19 1.97178099e-19]
 [3.58500223e-20 1.99064629e-20]
 [5.84214072e-21 2.15008188e-21]
 [9.42922962e-22 2.51928776e-22]
 [1.51395542e-22 3.21028195e-23]
 [2.42384843e-23 2.20160459e-23

In [128]:
backwardMatrix=backward(x, A, B)
print("backwardMatrix",backwardMatrix.T)

backwardMatrix [[2.27767058e-22 6.50636764e-22]
 [1.21735422e-21 7.00386141e-21]
 [5.26182864e-21 1.53692539e-20]
 [2.80036453e-20 1.65583628e-19]
 [1.19464231e-19 3.63539016e-19]
 [6.30641343e-19 3.92253697e-18]
 [2.62200254e-18 8.61963762e-18]
 [1.36152029e-17 1.86504835e-17]
 [7.99140593e-17 1.92428695e-16]
 [4.39773308e-16 4.11331421e-16]
 [2.64867816e-15 4.07985316e-15]
 [1.53869484e-14 8.49645339e-15]
 [9.47536297e-14 7.68580692e-14]
 [5.74837858e-13 7.47527091e-13]
 [3.38807801e-12 7.67843472e-12]
 [1.88142245e-11 8.18318257e-11]
 [9.06438407e-11 1.78491322e-10]
 [5.12858051e-10 1.88826320e-09]
 [2.59171119e-09 4.10015114e-09]
 [1.50178766e-08 8.55522933e-09]
 [9.23881939e-08 7.79491789e-08]
 [5.59425286e-07 1.52500942e-07]
 [3.50017207e-06 1.04627490e-06]
 [2.18671138e-05 7.57581118e-06]
 [1.36246672e-04 5.89448146e-05]
 [8.44763040e-04 4.98504784e-04]
 [5.19078704e-03 4.57768519e-03]
 [3.13611111e-02 4.50555556e-02]
 [1.83333333e-01 4.66666667e-01]
 [1.00000000e+00 1.00000000e

In [129]:
posteriorMatrix = posterior(forwardMatrix, backwardMatrix)
print("posteriorMatrix:",posteriorMatrix)

posteriorMatrix: [[0.4103509  0.70332221]
 [0.35822531 0.80757339]
 [0.25021521 1.02359359]
 [0.22638734 1.07124932]
 [0.15935485 1.2053143 ]
 [0.15061735 1.22278931]
 [0.10596261 1.31209878]
 [0.10439075 1.31524252]
 [0.14397698 1.23607005]
 [0.1489909  1.2260422 ]
 [0.20787007 1.10828386]
 [0.22603183 1.07196034]
 [0.32000876 0.88400649]
 [0.36248302 0.79905797]
 [0.36845386 0.78711628]
 [0.34002981 0.84396439]
 [0.26717328 0.98967746]
 [0.26304189 0.99794022]
 [0.22188255 1.0802589 ]
 [0.23654447 1.05093506]
 [0.32498132 0.87406137]
 [0.36384519 0.79633364]
 [0.51275354 0.49851693]
 [0.5940294  0.33596521]
 [0.63637429 0.25127542]
 [0.65474179 0.21454043]
 [0.65561812 0.21278776]
 [0.63931277 0.24539847]
 [0.6000677  0.32388862]
 [0.52402401 0.47597599]]
